In [ ]:
!pip install scispacy
!pip install spacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz

In [ ]:
import scispacy
import json
import spacy
from sklearn.utils import shuffle
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/

In [ ]:
%ls

## Data

In [ ]:
types = ('LOC', 'CORONAVIRUS', 'LIVESTOCK', 'WILDLIFE', 'EVOLUTION', 'PHYSICAL_SCIENCE',
'SUBSTRATE', 'MATERIAL', 'IMMUNE_RESPONSE')

In [ ]:
X = []

In [ ]:
with open('./SS-NER-prune.json') as file:
    for line in file:
        d = json.loads(line)
        X.append(d)

In [ ]:
test_X = X[:int(len(X)*(0.2))]
train_X = X[int(len(X)*(0.4)):]

## Spacy

### en_ner_bc5cdr

In [ ]:
from spacy.util import minibatch, compounding
import random
from spacy.training.example import Example

In [ ]:
bc5cdr = spacy.load("en_ner_bc5cdr_md")

In [ ]:
bc5cdr.pipe_names

In [ ]:
ner = bc5cdr.get_pipe('ner')

In [ ]:
for name in bc5cdr.pipe_names:
    if name != "ner":
        bc5cdr.disable_pipe(name)

In [ ]:
for i in types:
    ner.add_label(i)

In [ ]:
optimizer = ner.create_optimizer()

In [ ]:
new_train = []
for x in train_X:
    if 'body' in x.keys() and 'entities' in x.keys():
        text = x['body']
        tmp = []
        for i in range(len(x['entities'])):
            cur = x['entities'][i]
            tmp.append((cur['start'], cur['end'], cur['type']))
        new_train.append((text, {"entities": tmp}))

In [ ]:
n_iter = 1

In [ ]:
for itn in range(n_iter):
    random.shuffle(new_train)
    losses = {}
    batches = minibatch(new_train, size=compounding(4., 32., 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
            # Update the model with iterating each text
        for i in range(len(texts)):
            if len(texts[i]) > 100000:
              continue
            doc = bc5cdr.make_doc(texts[i])
            example.append(Example.from_dict(doc, annotations[i]))
        # Update the model
        bc5cdr.update(example, sgd=optimizer, drop=0.5, losses=losses)
        # ner.update(texts, annotations, sgd=optimizer, drop=0.35,
        #                 losses=losses)
            # doc = bc5cdr.make_doc(texts)
            # for a in annotations:
            #     example.append(Example.from_dict(doc, a))
                
            # Updating the weights
        # bc5cdr.update(example, sgd=optimizer, drop=0.35, losses=losses)
    print('Losses', losses)

In [ ]:
bc5cdr.meta['name'] = 'model60'  # rename model
bc5cdr.to_disk('./')

#### Evaluation

In [ ]:
from spacy.scorer import Scorer
from spacy.tokens import Doc
from spacy.training.example import Example

In [ ]:
def evaluate(ner_model, examples):
    scorer = Scorer()
    example = []
    for input_, annot in examples:
        pred = ner_model(input_)
        temp = Example.from_dict(pred, dict.fromkeys(annot))
        example.append(temp)
    scores = scorer.score(example)
    return scores

In [ ]:
scorer = Scorer(bc5cdr)

In [ ]:
new_test = []
for t in test_X:
    txt = t['body']
    et = {}
    lst = []
    for e in t['entities']:
        lst.append((e['start'],e['end'],e['type']))
    et['entities'] = lst
    example = Example.from_dict(bc5cdr.make_doc(txt), et)
    new_test.append(example)

In [ ]:
scores = scorer.score(new_test)

In [ ]:
evaluate(bc5cdr, new_test)

In [ ]:
pred_y = []
true_y = []
i = 0
for d in test_X:
    if i % 100 == 0: print(i)
    doc = bc5cdr(d['body'])
    tmp = []
    for e in doc.ents:
        tmp.append(e.text)
    pred_y.append(tmp)
    ground = []
    for e in d['entities']:
        ground.append(e['text'])
    true_y.append(ground)
    i += 1

In [ ]:
def f1(y_true, y_pred):
    i = list((Counter(y_true) & Counter(y_pred)).elements())
    # i = set(y_true).intersection(y_pred)
    print(len(i))
    recall = len(i) / len(y_true)
    precision = len(i) / len(y_pred)
    print('recall', recall)
    print('precision', precision)
    if recall + precision == 0:
        return 0
    else:
        return 2 * (precision * recall) / (precision + recall)


In [ ]:
def avgf1(true_y, pred_y):
    f_list = []
    for t, p in zip(true_y, pred_y):
        f_list.append(f1(t, p))
    return sum(f_list)/len(f_list)